# N-gram Language Models (12 + 10 + 10 pt)

In [ ]:
# load libraries
import nltk
from nltk.corpus import PlaintextCorpusReader

from nltk.util import ngrams
from nltk.lm.preprocessing import pad_both_ends

from tqdm import tqdm

# ngram:
_N = 3

In [ ]:
# Download a wikipedia dataset:
# ! wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
# ! unzip wikitext-2-raw-v1.zip

## Preprocessing

In [ ]:
# create a corpus reader
# this includes: sentence segmentation and word tokenization:
wikitext2 = PlaintextCorpusReader(
    'wikitext-2-raw',
    ['wiki.train.raw', 'wiki.valid.raw', 'wiki.test.raw'],
)
word_tokenizer = wikitext2._word_tokenizer

In [ ]:
# training and test split:
train = wikitext2.sents('wiki.train.raw')
test = wikitext2.sents('wiki.test.raw')

# the vocabulary based on the training data:
vocab = nltk.lm.Vocabulary([
    word
    for sent in train
    for word in sent
], unk_cutoff=1)

In [ ]:
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()

In [ ]:
# build n-grams
def build_ngrams(sent, n):
    # pad both ends for corner-ngrams:
    sent = ['<s>']*(n-1) + sent + ['</s>']*(n-1)
    # build the ngrams:
    return list(ngrams(sent, n))

In [ ]:
# run this cell to inspect how it works:
sample = "Minecraft is a sandbox video game developed by Mojang."
sample_tokinized = word_tokenizer.tokenize(sample)
sample_trigrams = build_ngrams(sample_tokinized, n=3)
print('sample_tokinized:')
print(sample_tokinized)
print('sample_trigrams')
print(sample_trigrams)


## Training the count model

In [ ]:
%%time
# compare these two models:
models = {
    'plain': nltk.lm.MLE, # plain count-based ngrams
    'smoothing': nltk.lm.Laplace, # with laplace smoothing
    'smoothing+interpolation': nltk.lm.KneserNeyInterpolated, # Modified Kneser & Ney 
}

for lm_name in models:
    # build and train the language model:
    models[lm_name] = models[lm_name](_N, vocabulary=vocab)

    # train on all n-grams (equal or lower order): N, N-1, ..., 1.
    for n in tqdm(range(_N, 0, -1), desc=lm_name):
        models[lm_name].fit([build_ngrams(sent, n) for sent in train])


#### Understand the models

In [ ]:
# Understand how fit words:
# fit() method builds all kinds of count dictionaries:
(
    models['plain'].counts[1], # unigrams
    models['plain'].counts[2], # bi-grams for conditional count freq (w_{t} | w_{t-1})
    models['plain'].counts[3], # tri-grams for conditional count freq (w_{t} | w_{t-2} w_{t-1})
)

In [ ]:
# for example: 
# Count( word_3='numer'   | word_1 = 'A', word_2 ='large' ) = 4
# Count( word_3='variety' | word_1 = 'A', word_2 ='large' ) = 3
# ...
list(models['plain'].counts[3].items())[200]

In [ ]:
# understand this:
models['plain'].counts[3][('A', 'large')]['number'] / sum(models['plain'].counts[3][('A', 'large')].values())

In [ ]:
models['plain'].score('number', ('A', 'large'))
# more details in chapter 3 equation 3.12.
# https://web.stanford.edu/~jurafsky/slp3/3.pdf

In [ ]:
# You can use the plain model for random language generation:
models['plain'].generate(10)

## Testing

In [ ]:
# Inspect log probabilities:
models['plain'].logscore('mind')

In [ ]:
sample = "Minecraft is a sandbox video game developed by Mojang."
sample_ngrams = [
    None,
    build_ngrams(word_tokenizer.tokenize(sample), n=1), # unigrams
    build_ngrams(word_tokenizer.tokenize(sample), n=2), # bigrams
    build_ngrams(word_tokenizer.tokenize(sample), n=3), # trigrams
]

In [ ]:
for model_name in models:
    print(f"{model_name} model:")
    for n in range(1, _N+1):
        print(f"{n}-gram", models[model_name].perplexity(sample_ngrams[n]))
    print()

### Questions

1. Why these models have `<UNK>` token? What is the log-probability of <UNK> in three models? (3pt)


These models have `<UNK>` so that 

plain: -inf, smoothing: -21.03873951979207, smoothing + interpolation -16.21348398616622

In [ ]:
models['plain'].logscore('<UNK>')

In [ ]:
models['smoothing'].logscore('<UNK>')

In [ ]:
models['smoothing+interpolation'].logscore('<UNK>')

2. Why plain count-based MLE model fails to produce perplexities? What are the possible solutions for it? (3pt)


In [ ]:
# anwer here (in English)

3. Show with an example why Laplace smoothing can produce perplexity for unseen words? (3pt)

In [ ]:
# anwer here (in English and python)

4. Why perplexity of bi-grams are lower than unigrams? (3pt)

In [ ]:
# anwer here (in English and python)
# use models['smoothing'].counts[2] to show how?

## Optional 1: measure perplexity of conditional trigrams (10pt)

The neural network below is based on Bengio et al. (2003). It is trained on moving windows described in chapter 9 figure 9.1 but with trigrams instead of 4-grams.
https://web.stanford.edu/~jurafsky/slp3/9.pdf

You don't need to train the model. However, a stand alone python code is provided in `bengio_lm.py` if you want to try training it on GPU.

Read the code below then report the perplexity of the language model on the sample sentence.

In [ ]:
import torch # neural network framework

# encoding the tokens:
vocab_list = [word for word, freq in vocab.counts.most_common() if freq > 1]
word2idx = {word: idx for idx, word in enumerate(['<s>', '</s>', vocab.unk_label]+vocab_list)}
idx2word = {idx: word for idx, word in enumerate(['<s>', '</s>', vocab.unk_label]+vocab_list)}

def token_encoder(tokens):
    if type(tokens) in {list, tuple}:
        return [word2idx[token] if token in word2idx else word2idx[vocab.unk_label] for token in tokens]
    elif type(tokens) == str:
        token = tokens
        return word2idx[token] if token in word2idx else word2idx[vocab.unk_label]
    print(type(tokens))

# moving window language model:
# https://jmlr.org/papers/volume3/tmp/bengio03a.pdf
class BengioLM(torch.nn.Module):
    def __init__(self, context_size=2, dim=50):
        super(BengioLM, self).__init__()
        # defining the parameters of the model
        self.C = torch.nn.Embedding(len(word2idx), dim) # C
        self.Hx_d = torch.nn.Linear(context_size*dim, dim) # d, H
        self.tanh = torch.nn.Tanh()
        self.Wx_Uf_b = torch.nn.Linear((context_size + 1) * dim, len(word2idx)) # b, U, W
        self.logsoftmax = torch.nn.LogSoftmax(dim=1)
        self.loss_fn = torch.nn.NLLLoss() # negative-log-likelihood loss
    
    def forward(self, context, target_idx=None):
        # function of the model
        batch_size = context.shape[0]
        x = self.C(context).view(batch_size,-1)
        x = torch.cat([x, self.tanh(self.Hx_d(x))], dim=-1)
        logprob = self.logsoftmax(self.Wx_Uf_b(x))
        
        if target_idx is None:
            return logprob
        else:
            loss = self.loss_fn(logprob, target_idx)
            return logprob, loss


#### The model is trained with Stochastic Gradient Descent with 10 epochs (skip this):

#### Load the model:

In [ ]:
# we ran the training code above on GPU and saved it in model.pt.
# load the pre-trained language model:
device = torch.device('cpu')
model = BengioLM()
model.load_state_dict(torch.load('model.pt', map_location=device))

In [ ]:
# this is how you can get the conditional log-probabilities of all words in the sentence
# P(target | w0, w1):
for w0, w1, target in build_ngrams(word_tokenizer.tokenize(sample), n=3):
    logprobs = model.forward(torch.tensor([token_encoder([w0,w1])]))
    print(target, logprobs[0, token_encoder(target)])

Write a code here to report Perplexity of the sample sentence.

For more information got to chapter 3, section 3.2.1 and chapter 9, equation 9.12.

https://web.stanford.edu/~jurafsky/slp3/3.pdf

https://web.stanford.edu/~jurafsky/slp3/9.pdf

In [ ]:
# perplexity of a sentence 
# code here

### Optional 2: implement a generate function using pre-trained language model above (10pt)


In [ ]:
# code here